---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    
    return spam_data['target'].mean()*100

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    
    # Extracts the tokens of the text (vocabulary as list)
    tokens = CountVectorizer().fit(X_train)
    
    # Creates matrix with texts as rows (records) and tokens as columns (features)
    X_train_tokenized = tokens.transform(X_train)
    
    return max(tokens.get_feature_names(),key = len)

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    
    # Extracts the tokens of the text (vocabulary as list)
    tokens = CountVectorizer().fit(X_train)
    
    # Creates matrix with texts as rows (records) and tokens as columns (features)
    X_train_tokenized = tokens.transform(X_train)
    
    # Model Fitting
    clf = MultinomialNB(alpha=0.1)
    clf.fit(X_train_tokenized,y_train)
    
    y_predicted = clf.predict(tokens.transform(X_test))
    
    return roc_auc_score(y_test,y_predicted)

In [8]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    # Extracts the tokens of the text (vocabulary as list) but with tfidf
    tf_token_model = TfidfVectorizer(min_df = 1).fit(X_train)
    tf_tokens = tf_token_model.get_feature_names()
    
    # Creates matrix with texts as rows (records) and tokens as columns (features)
    X_train_tokenized = tf_token_model.transform(X_train)
    
    # TDIDF values, tokens (feature names), index of ascending sorted array by tfidf values 
    tfidf_values = X_train_tokenized.max(axis=0).toarray()[0]
    feature_names = np.array(tf_tokens)
    sorted_index = X_train_tokenized.max(axis=0).toarray()[0].argsort()
    
    
    df = pd.DataFrame(data = {'tokens': feature_names,'tfidf_values': tfidf_values})
    
    # Smallest
    df = df.sort_values(by=list(df.columns)) # Ascending
    smallest_series = pd.Series(index = df['tokens'].values, data = df['tfidf_values'].values )[:20] 
    
    # Largest
    df = df.sort_values(by=list(df.columns),ascending = [False,True]) # Descending
    largest_series = pd.Series(index = df['tokens'].values, data = df['tfidf_values'].values )[:20]

    return (smallest_series,largest_series)


In [10]:
#answer_four().shape
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 afternoons      0.091250
 approaching     0.091250
 dtype: float64, 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [11]:
def answer_five():
    
    # Extracts the tokens of the text (vocabulary as list) but with tfidf
    tf_token_model = TfidfVectorizer(min_df = 3).fit(X_train)
    tf_tokens = tf_token_model.get_feature_names()
    
    # Creates matrix with texts as rows (records) and tokens as columns (features)
    X_train_tokenized = tf_token_model.transform(X_train)
    
    # Model Fitting
    clf = MultinomialNB(alpha=0.1)
    clf.fit(X_train_tokenized,y_train)
    
    y_predicted = clf.predict(tf_token_model.transform(X_test))
    
    return roc_auc_score(y_test,y_predicted)

In [12]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [13]:
def answer_six():
    
    avg_spam = spam_data[ spam_data['target'] == 1 ]['text'].apply(len).mean()
    avg_not_spam = spam_data[ spam_data['target'] == 0 ]['text'].apply(len).mean()
    
    return (avg_not_spam,avg_spam)

In [14]:
answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [16]:
from sklearn.svm import SVC

def answer_seven():
    
     # Extracts the tokens of the text (vocabulary as list) but with tfidf
    tf_token_model = TfidfVectorizer(min_df = 5).fit(X_train)
    tf_tokens = tf_token_model.get_feature_names()
    
    # Creates matrix with texts as rows (records) and tokens as columns (features)
    X_train_tokenized = tf_token_model.transform(X_train)
    
    # Adding new feature
    new_feature = X_train.apply(len)
    X_train_tokenized = add_feature(X_train_tokenized,new_feature)
    X_test_tokenized = add_feature(tf_token_model.transform(X_test), X_test.apply(len))
    
    # Model fitting
    clf = SVC(C=10e5)
    clf.fit(X_train_tokenized,y_train)
    
    y_predicted = clf.predict(X_test_tokenized)
    
    
    return roc_auc_score(y_test,y_predicted)

In [17]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [18]:
def answer_eight():
    import re
    
    avg_spam = spam_data[spam_data['target'] == 1 ]['text'].apply(lambda row: len(re.findall(r'\d',row))).mean()
    avg_not_spam = spam_data[spam_data['target'] == 0 ]['text'].apply(lambda row: len(re.findall(r'\d',row))).mean()
    
    return (avg_not_spam,avg_spam)

In [19]:
answer_eight()

(0.29927461139896372, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [20]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    
    import re
     # Extracts the tokens of the text (vocabulary as list) but with tfidf
    tf_token_model = TfidfVectorizer(min_df = 5, ngram_range=(1,3)).fit(X_train)
    tf_tokens = tf_token_model.get_feature_names()
    
    # Creates matrix with texts as rows (records) and tokens as columns (features)
    X_train_tokenized = tf_token_model.transform(X_train)
    X_test_tokenized = tf_token_model.transform(X_test)
    
    # Adding new feature
    
    # Training
    n_digits = X_train.apply(lambda row: len(re.findall(r'\d',row)))
    new_features = [X_train.apply(len), n_digits ]
    X_train_tokenized = add_feature(X_train_tokenized,new_features)
    
    #Test
    n_digits = X_test.apply(lambda row: len(re.findall(r'\d',row)))
    new_features = [X_test.apply(len), n_digits]
    X_test_tokenized = add_feature(X_test_tokenized, new_features)
    
    # Model fitting
    clf = LogisticRegression(C=100)
    clf.fit(X_train_tokenized,y_train)
    
    y_predicted = clf.predict(X_test_tokenized)
    
    
    return roc_auc_score(y_test,y_predicted)

In [21]:
answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [22]:
def answer_ten():
    import re
    
    avg_spam = spam_data[spam_data['target'] == 1 ]['text'].apply(lambda row: len(re.findall(r'\W',row))).mean()
    avg_not_spam = spam_data[spam_data['target'] == 0 ]['text'].apply(lambda row: len(re.findall(r'\W',row))).mean()
    
    return (avg_not_spam,avg_spam)

In [23]:
answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [26]:
def answer_eleven():
    
    import pandas as pd
    import re
    # Extracts the tokens of the text (vocabulary as list) but with tfidf
    token_model = CountVectorizer(min_df = 5, ngram_range=(2,5),analyzer = 'char_wb').fit(X_train)
    tokens = token_model.get_feature_names()
    
    # Creates matrix with texts as rows (records) and tokens as columns (features)
    X_train_tokenized = token_model.transform(X_train)
    X_test_tokenized = token_model.transform(X_test)
    
    # Adding new features
    
    new_features_names = ['length_of_doc', 'digit_count', 'non_word_char_count']
    # Training
    n_digits = X_train.apply(lambda row: len(re.findall(r'\d',row)))
    n_non_words = X_train.apply(lambda row: len(re.findall(r'\W',row)))
    new_features = [X_train.apply(len), n_digits,n_non_words]
    X_train_tokenized = add_feature(X_train_tokenized,new_features)
    
    #Test
    n_digits = X_test.apply(lambda row: len(re.findall(r'\d',row)))
    n_non_words = X_test.apply(lambda row: len(re.findall(r'\W',row)))
    new_features = [X_test.apply(len), n_digits,n_non_words]
    X_test_tokenized = add_feature(X_test_tokenized, new_features)
    
    
    # Model fitting
    clf = LogisticRegression(C=100)
    clf.fit(X_train_tokenized,y_train)
    
    y_predicted = clf.predict(X_test_tokenized)
    roc_score = roc_auc_score(y_test,y_predicted)
    
    # Features & Coefficients
    coefs = clf.coef_[0]
    
    features = tokens + new_features_names
    features_df = pd.DataFrame(data = {'features': features , 'coefficients': coefs}).sort_values(by='coefficients')
    
    smallest_series = features_df.set_index('features')['coefficients'][:10]
    largest_series = features_df.set_index('features')['coefficients'][-10:][::-1]
    
    return (roc_score,smallest_series,largest_series)

In [27]:
answer_eleven()

(0.97885931107074342, features
 .     -0.869753
 ..    -0.860883
 ?     -0.676982
  i    -0.667004
  y    -0.614901
  go   -0.579590
 :)    -0.535074
  h    -0.505766
 go    -0.498507
  m    -0.490960
 Name: coefficients, dtype: float64, features
 digit_count    1.212230
 ne             0.597778
 ia             0.541483
 co             0.538766
 xt             0.521487
  ch            0.520351
 mob            0.517868
  x             0.516096
 ww             0.508671
 ar             0.502644
 Name: coefficients, dtype: float64)